# STL to block mesh dictionary python script 
### 12/26/22
### update 1/4/23

In [1]:
from __future__ import unicode_literals, print_function

import math
import numpy as np
from ofblockmeshdicthelper import BlockMeshDict, Vertex, SimpleGrading


In [ ]:
# ^ z
# |  /y (to the outlet )
# | /
# |/_________> x    


# ^ z
# |
# |
# |__________________________> y

In [2]:
# take in box parameters and build a mesh around them 
box_params = { #m
    "x": 4.65, # x-z plane faces front of the wind tunnel, width 
    "y": 3.65, # depth of the box 
    "z": 3.66 # height 0 vertical 
}

## from block mesh dict 

### wind tunnel 

In [4]:
# ramponi + blocken 2012        
# * H_D = H + d, 
# * W_D = W + 2d
# * H, W = height and width of the building respectively 
# * d = 5H - 3H (start with 3H for early study) 
# * Upstream, downstream length => 5H and 15H 

H = box_params["z"]
d = 3 * H # 3H 

wt_params = {
    "x": box_params["x"] + 2*d,   # width / length
    "y": box_params["y"] + 5*H + 15*H, # depth 
    "z": box_params["z"] + d, # height 
}
wt_params


{'x': 26.61, 'y': 76.85000000000001, 'z': 14.64}

In [13]:
# prepare ofblockmeshdicthelper.BlockMeshDict instance to
# gather vertices, blocks, faces and boundaries.
bmd = BlockMeshDict()

# set metrics
bmd.set_metric('m')


In [14]:
# wind tunnel 

basevs = [
    Vertex(0, 0, 0, 'v0'),
    Vertex(wt_params["x"], 0, 0, 'v1'),
    Vertex(wt_params["x"],  wt_params["y"], 0, 'v2'),
    Vertex(0, wt_params["y"], 0, 'v3')]

# for v in basevs:
#     print(f"{v.name}: {v.x, v.y, v.z}")


# raise up in the z direction 
for v in basevs:
    bmd.add_vertex(v.x, v.y, wt_params["z"], v.name+'+z')
    bmd.add_vertex(v.x, v.y, 0, v.name+'-z')

for v in basevs:
    print(f"{v.name}: {v.x, v.y, v.z}")

v0: (0, 0, 0)
v1: (26.61, 0, 0)
v2: (26.61, 76.85000000000001, 0)
v3: (0, 76.85000000000001, 0)


In [32]:
# print names and coordinates of a block mesh dict object 
def print_vertices(bmd):
    for k,v in bmd.vertices.items():
        print(f"{k} : ({bmd.vertices[k].x}, {bmd.vertices[k].y}, {bmd.vertices[k].z})")

In [10]:
def vnamegen(x0y0, x1y0, x1y1, x0y1):
    "generate names from the hex block"
    return (x0y0+'-z', x1y0+'-z', x1y0+'+z', x0y0+'+z',
            x0y1+'-z', x1y1+'-z', x1y1+'+z', x0y1+'+z')

In [11]:
vnamegen('v0', 'v1', 'v2', 'v3')

('v0-z', 'v1-z', 'v1+z', 'v0+z', 'v3-z', 'v2-z', 'v2+z', 'v3+z')

In [34]:
b0 = bmd.add_hexblock(vnamegen('v0', 'v1', 'v2', 'v3'),
                      (80, 50, 20),
                      'b0',
                      grading=SimpleGrading(1,
                                            1,
                                            1))

v0+z : (0, 0, 14.64)
v0-z : (0, 0, 0)
v1+z : (26.61, 0, 14.64)
v1-z : (26.61, 0, 0)
v2+z : (26.61, 76.85000000000001, 14.64)
v2-z : (26.61, 76.85000000000001, 0)
v3+z : (0, 76.85000000000001, 14.64)
v3-z : (0, 76.85000000000001, 0)


In [ ]:
        # index is number or keyword to identify the face of Hex
        #     0 = 'w' = 'xm' = '-100' = (0 4 7 3)
        #     1 = 'e' = 'xp' = '100' = (1 2 5 6)
        #     2 = 's' = 'ym' = '0-10' = (0 1 5 4)
        #     3 = 'n' = 'yp' = '010' = (2 3 7 6)
        #     4 = 'b' = 'zm' = '00-1' = (0 3 2 1)
        #     5 = 't' = zp' = '001' = (4 5 6 7)
        # name is given to Face instance. If omitt

In [48]:
print_vertices(bmd)

v0+z : (0, 0, 14.64)
v0-z : (0, 0, 0)
v1+z : (26.61, 0, 14.64)
v1-z : (26.61, 0, 0)
v2+z : (26.61, 76.85000000000001, 14.64)
v2-z : (26.61, 76.85000000000001, 0)
v3+z : (0, 76.85000000000001, 14.64)
v3-z : (0, 76.85000000000001, 0)


In [47]:
for i in range(6):
    print(b0.face(i).vnames)

('v0-z', 'v3-z', 'v3+z', 'v0+z')
('v1-z', 'v1+z', 'v2+z', 'v2-z')
('v0-z', 'v1-z', 'v2-z', 'v3-z')
('v1+z', 'v0+z', 'v3+z', 'v2+z')
('v0-z', 'v0+z', 'v1+z', 'v1-z')
('v3-z', 'v2-z', 'v2+z', 'v3+z')


In [49]:
boundaries = ["symmetry", "symmetry", "wall", "patch", "patch", "patch"]
names = ["side01", "side02", "bottom", "top", "inlet", "outlet"]
for i, boundary, name in zip(range(6), boundaries, names):
    bmd.add_boundary(boundary, name, [b0.face(i)])

In [57]:
for k,v in bmd.boundaries.items():
    print(f"{k}: {bmd.boundaries[k].faces[0].vnames}")
    

side01: ('v0-z', 'v3-z', 'v3+z', 'v0+z')
side02: ('v1-z', 'v1+z', 'v2+z', 'v2-z')
bottom: ('v0-z', 'v1-z', 'v2-z', 'v3-z')
top: ('v1+z', 'v0+z', 'v3+z', 'v2+z')
inlet: ('v0-z', 'v0+z', 'v1+z', 'v1-z')
outlet: ('v3-z', 'v2-z', 'v2+z', 'v3+z')


### internal block 

In [ ]:
# now want internal block
# base vertices of wind tunnel 
boxvs = [
    Vertex(box_size, 0, box_size, 'v4'),
    Vertex(box_size + box_size, 0, box_size, 'v5'),
    Vertex(box_size + box_size,  0, box_size + box_size, 'v6'),
    Vertex(box_size, 0, box_size + box_size, 'v7')]

In [ ]:
# raise up in the y direction 
for v in boxvs:
    bmd.add_vertex(v.x, box_size, v.z, v.name+'+y')
    bmd.add_vertex(v.x, 0, v.z, v.name+'-y')

In [ ]:
b1 = bmd.add_hexblock(vnamegen('v4', 'v5', 'v6', 'v7'),
                      (2, 5, 5),
                      'b1',
                      grading=SimpleGrading(1,
                                            1,
                                            1))
[b1.face(i).name for i in range(6)]

In [ ]:
bmd.add_boundary('wall', 'box', [b1.face(i) for i in range(6) ]) 

# Finish 

In [ ]:
bmd.assign_vertexid()

In [ ]:
print(bmd.format())